Exemple : Partitionner par departement

In [0]:
print("PARTITIONNEMENT PAR DEPARTEMENT")
print("=" * 70)

# Charger les donnees Silver
df_silver = spark.table("gares_silver")

# Sauvegarder avec partitionnement par departement
df_silver.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("code_departement") \
    .saveAsTable("gares_silver_partitioned")

print("Table partitionnee creee : gares_silver_partitioned")

# Verifier la structure
print("\nStructure des fichiers :")
files = dbutils.fs.ls("dbfs:/user/hive/warehouse/gares_silver_partitioned")
for f in files[:10]:  # Afficher les 10 premiers
    print(f"  {f.name}")

PARTITIONNEMENT PAR DEPARTEMENT
Table partitionnee creee : gares_silver_partitioned

Structure des fichiers :


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-7582471938144712>, line 18
     16 # Verifier la structure
     17 print("\nStructure des fichiers :")
---> 18 files = dbutils.fs.ls("dbfs:/user/hive/warehouse/gares_silver_partitioned")
     19 for f in files[:10]:  # Afficher les 10 premiers
     20     print(f"  {f.name}")

File /databricks/python_shell/lib/dbruntime/remotefshandler/RemoteFsHandler.py:55, in prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
     52 class ExecutionError(Exception):
     53     pass
---> 55 raise ExecutionError(str(e)) from None

ExecutionError: (java.io.FileNotFoundException) No such file or directory dbfs:/user/hive/warehouse/gares_silver_partitioned

JVM stacktrace:
java.io.FileNotFoundException
	at com.databricks.backend.daemon.data.client.DBFSV2.$anonfun$listStatus$2(DatabricksFileSystemV2.scala:194)
	at 

Avantage du partitionnement

In [0]:
print("COMPARAISON DE PERFORMANCES")
print("=" * 70)

import time

# Test 1 : Sans partitionnement
df_non_partitioned = spark.table("gares_silver")

start = time.time()
result1 = df_non_partitioned.filter(col("code_departement") == "75").count()
temps1 = time.time() - start

print(f"Sans partitionnement : {result1} lignes en {temps1:.3f} secondes")

# Test 2 : Avec partitionnement
df_partitioned = spark.table("gares_silver_partitioned")

start = time.time()
result2 = df_partitioned.filter(col("code_departement") == "75").count()
temps2 = time.time() - start

print(f"Avec partitionnement : {result2} lignes en {temps2:.3f} secondes")

print(f"\nGain de performance : {(temps1/temps2):.2f}x plus rapide")